In [ ]:
%matplotlib inline
import numpy as np
import pandas as pd
import tensorflow as tf

from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers, losses
from tensorflow.keras.models import Model
from astropy.timeseries import LombScargle
import random
from tensorflow.keras.layers import Dense, Flatten, Conv2D, Input
from tensorflow.keras import Model
import matplotlib.pyplot as plt

In [ ]:
!wget https://zenodo.org/record/2539456/files/plasticc_train_metadata.csv.gz?download=1 -O plasticc_train_metadata.csv.gz
!wget https://zenodo.org/record/2539456/files/plasticc_train_lightcurves.csv.gz?download=1 -O plasticc_train_lightcurves.csv.gz
meta = pd.read_csv("plasticc_train_metadata.csv.gz").set_index("object_id")
lc = pd.read_csv("plasticc_train_lightcurves.csv.gz").set_index(["object_id"])


--2022-06-23 15:50:05--  https://zenodo.org/record/2539456/files/plasticc_train_metadata.csv.gz?download=1
Resolving zenodo.org (zenodo.org)... 137.138.76.77
Connecting to zenodo.org (zenodo.org)|137.138.76.77|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 370350 (362K) [application/octet-stream]
Saving to: ‘plasticc_train_metadata.csv.gz’

plasticc_train_meta 100%[===================>] 361.67K  1.02MB/s    in 0.3s    

2022-06-23 15:50:07 (1.02 MB/s) - ‘plasticc_train_metadata.csv.gz’ saved [370350/370350]

--2022-06-23 15:50:08--  https://zenodo.org/record/2539456/files/plasticc_train_lightcurves.csv.gz?download=1
Resolving zenodo.org (zenodo.org)... 137.138.76.77
Connecting to zenodo.org (zenodo.org)|137.138.76.77|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 21553100 (21M) [application/octet-stream]
Saving to: ‘plasticc_train_lightcurves.csv.gz’

plasticc_train_ligh 100%[===================>]  20.55M  12.6MB/s    in 1.6s    

2

In [ ]:
!wget https://zenodo.org/record/2539456/files/plasticc_test_metadata.csv.gz?download=1 -O plasticc_test_metadata.csv.gz
!wget https://zenodo.org/record/2539456/files/plasticc_test_lightcurves_02.csv.gz?download=1 -O plasticc_test_lightcurves.csv.gz


test_meta = pd.read_csv("plasticc_test_metadata.csv.gz").set_index("object_id")
test_lc = pd.read_csv("plasticc_test_lightcurves.csv.gz").set_index(["object_id"])

--2022-06-23 15:50:32--  https://zenodo.org/record/2539456/files/plasticc_test_metadata.csv.gz?download=1
Resolving zenodo.org (zenodo.org)... 137.138.76.77
Connecting to zenodo.org (zenodo.org)|137.138.76.77|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 157245185 (150M) [application/octet-stream]
Saving to: ‘plasticc_test_metadata.csv.gz’

plasticc_test_metad 100%[===================>] 149.96M  20.8MB/s    in 8.9s    

2022-06-23 15:50:42 (16.9 MB/s) - ‘plasticc_test_metadata.csv.gz’ saved [157245185/157245185]

--2022-06-23 15:50:42--  https://zenodo.org/record/2539456/files/plasticc_test_lightcurves_02.csv.gz?download=1
Resolving zenodo.org (zenodo.org)... 137.138.76.77
Connecting to zenodo.org (zenodo.org)|137.138.76.77|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 710972039 (678M) [application/octet-stream]
Saving to: ‘plasticc_test_lightcurves.csv.gz’

plasticc_test_light 100%[===================>] 678.04M  21.0MB/s    in 33

In [ ]:
category_mapping = {90: 'SN1a', 67: 'SN1a-91bg', 52: 'SN1ax', 42: 'SN2', 62: 'SN1bc', 95: 'SLSN1', 15: 'TDE', 64: 'KN', 88: 'AGN', 92: 'RRL', 65: 'M-dwarf', 16: 'EB', 53: 'Mira', 6: 'Microlens', 99: 'unknown'}
new_cats = {}
for i, label in enumerate(category_mapping.keys()):
  new_cats[label] =  i
categories_to_use = [key for key in category_mapping.keys()]
#print(new_cats)

In [ ]:

sn_categories = [90, 67, 52, 42, 95, 62]




In [ ]:
def feature_normalization(feature_values, return_norm=True):
  norm_layer = tf.keras.layers.Normalization(axis=None)
  norm_layer.adapt(feature_values)
  normed_feature = norm_layer(feature_values)
  if return_norm:
    return normed_feature, norm_layer
  else:
    return normed_feature


def select_flux_band(df, band):
  time = df[df['passband'] == band]['mjd']
  flux = df[df['passband'] == band]['flux']
  err = df[df['passband'] == band]['flux_err']
  return time, flux, err

In [ ]:
all_sources = {}
for id in lc.index.unique():
  source_df = pd.DataFrame(data = lc[lc.index == id]).reset_index()
  source_df['ra'] = meta.loc[id, 'ra']
  source_df['dec'] = meta.loc[id, 'decl']
  target = meta.loc[id, 'target']
  source_df['target'] = target
  source_df['category'] = new_cats[int(meta.loc[id, 'target'])]
  source_df['spec_redshift'] = meta.loc[id, 'hostgal_specz']
  all_sources[id] = source_df

In [ ]:
all_sources = {}

for id in lc.index.unique():
  source_df = pd.DataFrame(data = lc[lc.index == id]).reset_index()
  source_df['ra'] = meta.loc[id, 'ra']
  source_df['dec'] = meta.loc[id, 'decl']
  target = meta.loc[id, 'target']
  source_df['target'] = target
  if target in sn_categories:
    source_df['category'] = 1
  else:
    source_df['category'] = 0
  source_df['spec_redshift'] = meta.loc[id, 'hostgal_specz']
  all_sources[id] = source_df

In [ ]:
#discriminate all classes


all_test_sources = {}
sample_size = 5000
just_some_source_ids = pd.Series(test_lc.index.unique()).sample(sample_size)


for id in just_some_source_ids:
  source_df = pd.DataFrame(data = test_lc[test_lc.index == id]).reset_index()
  source_df['ra'] = test_meta.loc[id, 'ra']
  source_df['dec'] = test_meta.loc[id, 'decl']
  source_df['target'] = test_meta.loc[id, 'true_target']
  true_target = test_meta.loc[id, 'true_target']
  if true_target > 100:
    true_target = 99
  source_df['category'] = new_cats[true_target]
  source_df['spec_redshift'] = test_meta.loc[id, 'hostgal_specz']
  all_test_sources[id] = source_df

In [ ]:
#discriminate just SN from non-SN


all_test_sources = {}

sample_size = 5000
just_some_source_ids = pd.Series(test_lc.index.unique()).sample(sample_size)


for id in just_some_source_ids:
  source_df = pd.DataFrame(data = test_lc[test_lc.index == id]).reset_index()
  source_df['ra'] = test_meta.loc[id, 'ra']
  source_df['dec'] = test_meta.loc[id, 'decl']
  true_target = test_meta.loc[id, 'true_target']
  source_df['target'] = true_target
  if true_target in sn_categories:
    source_df['category'] = 1
  else:
    source_df['category'] = 0
  source_df['spec_redshift'] = test_meta.loc[id, 'hostgal_specz']
  all_test_sources[id] = source_df

In [ ]:
extracted_features = pd.DataFrame()



for id, df in all_sources.items():
  redsh = df.loc[0, "spec_redshift"]
  extracted_features.loc[id, 'z'] = redsh
  global_min_flux = min(df['flux'].min(), 0)
  flux_shift = abs(global_min_flux)
  band_info = []
  for i in range(6):
    time, flux, err = select_flux_band(df, i)
    band_flux = flux + flux_shift
    flux_mean = np.mean(band_flux)
    flux_std = np.std(band_flux)
    flux_measurements = len(band_flux)

    extracted_features.loc[id, f"flux_{i}_mean"] = flux_mean
    extracted_features.loc[id, f"flux_{i}_std"] = flux_std
    extracted_features.loc[id, f"flux_{i}_n"] = flux_measurements

In [ ]:
features_to_exclude = ['z']

In [ ]:
normalized_features = pd.DataFrame(index = extracted_features.index)#, columns = extracted_features.columns)


feature_norms = {}

for col in extracted_features.columns:
  if col in features_to_exclude:
    continue
  normed_feature, feature_norm = feature_normalization(extracted_features[col].values)
  normalized_features[col] = normed_feature
  feature_norms[col] = feature_norm

In [ ]:
normalized_features.head()

,flux_0_mean,flux_0_std,flux_0_n,flux_1_mean,flux_1_std,flux_1_n,flux_2_mean,flux_2_std,flux_2_n,flux_3_mean,flux_3_std,flux_3_n,flux_4_mean,flux_4_std,flux_4_n,flux_5_mean,flux_5_std,flux_5_n
615,0.030759,-0.011622,1.655837,0.007699,0.374669,1.860865,0.025270,0.133995,1.902508,0.026119,0.063120,1.891318,0.032146,0.042121,1.908384,0.032855,0.027943,1.657435
713,-0.039373,-0.018657,1.962188,-0.044624,-0.157400,1.756058,-0.045225,-0.104007,1.764982,-0.045309,-0.085755,1.757394,-0.044609,-0.092337,1.740165,-0.044221,-0.076198,1.569731
730,-0.038914,-0.019142,2.049716,-0.044207,-0.160886,1.546444,-0.044661,-0.104121,1.489929,-0.044670,-0.084969,1.489546,-0.043891,-0.090315,1.319617,-0.043424,-0.073944,1.131208
745,-0.039032,-0.018908,2.049716,-0.044065,-0.139328,1.756058,-0.044391,-0.090141,1.764982,-0.044114,-0.072853,1.757394,-0.043507,-0.079780,1.740165,-0.043236,-0.069335,1.482026
1124,-0.039038,-0.019093,1.655837,-0.044067,-0.155260,1.860865,-0.044274,-0.095770,1.902508,-0.044282,-0.076784,1.891318,-0.043669,-0.082697,1.908384,-0.043443,-0.071007,1.657435


In [ ]:
full_train_x = normalized_features.values
full_train_labels = np.asarray([all_sources[id].loc[0, 'category'] for id in all_sources])

x_train, x_test, y_train, y_test = train_test_split(full_train_x, full_train_labels, test_size = 0.15, random_state = 42)



In [ ]:
extracted_features_test_data = pd.DataFrame(index = all_test_sources.keys())

for id, df in all_test_sources.items():
  redsh = df.loc[0, "spec_redshift"]
  extracted_features_test_data.loc[id, 'z'] = redsh
  global_min_flux = min(df['flux'].min(), 0)
  flux_shift = abs(global_min_flux)
  band_info = []
  for i in range(6):
    time, flux, err = select_flux_band(df, i)
    band_flux = flux + flux_shift
    flux_mean = np.mean(band_flux)
    flux_std = np.std(band_flux)
    flux_measurements = len(band_flux)

    extracted_features_test_data.loc[id, f"flux_{i}_mean"] = flux_mean
    extracted_features_test_data.loc[id, f"flux_{i}_std"] = flux_std
    extracted_features_test_data.loc[id, f"flux_{i}_n"] = flux_measurements

In [ ]:
print(np.sum(y_train))
print(len(y_train))

3876
6670


In [ ]:
# normalize test set individually

normalized_test_features = pd.DataFrame(index = extracted_features_test_data.index)#, columns = extracted_test_features.columns)
for col in extracted_features_test_data.columns:
  if col not in features_to_exclude:
    normalized_test_features[col] = feature_normalization(extracted_features_test_data[col].values, return_norm=False)




In [ ]:
# use same normalization as in training data set

normalized_test_features = pd.DataFrame(index = extracted_features_test_data.index)#, columns = extracted_test_features.columns)

for col in feature_norms:
  norm_layer = feature_norms[col]
  normalized_test_features[col] = norm_layer(extracted_features_test_data[col].values)

In [ ]:
normalized_test_features.head()

,flux_0_mean,flux_0_std,flux_0_n,flux_1_mean,flux_1_std,flux_1_n,flux_2_mean,flux_2_std,flux_2_n,flux_3_mean,flux_3_std,flux_3_n,flux_4_mean,flux_4_std,flux_4_n,flux_5_mean,flux_5_std,flux_5_n
10401218,-0.035436,-0.017342,-0.532385,-0.040259,-0.148553,-0.654503,-0.041040,-0.100356,-0.572967,-0.041107,-0.083276,-0.720201,-0.040445,-0.088788,-0.783126,-0.040080,-0.067767,-1.149110
5015664,-0.036259,-0.018514,-0.357327,-0.041214,-0.160294,-0.444889,-0.041597,-0.100561,-0.091624,-0.041581,-0.082747,-0.184505,-0.041127,-0.089407,-0.867236,-0.040017,-0.068560,-0.008951
2271912,-0.036440,-0.018725,-0.576149,-0.041589,-0.155123,-0.497292,-0.042104,-0.103157,-0.091624,-0.042308,-0.084228,-0.050581,-0.041664,-0.089274,-0.446687,-0.040399,-0.067671,-0.008951
13433705,-0.024521,0.025723,-0.532385,-0.040901,-0.085758,-0.444889,-0.038300,0.021730,-0.297914,-0.037429,0.026188,-0.452353,-0.033281,0.040670,-0.278468,-0.031243,0.042816,-0.447474
4529379,-0.003096,-0.016049,-0.663678,-0.005348,-0.060339,-0.602099,-0.007082,-0.019939,-0.779256,-0.010880,0.001435,-0.988049,-0.005855,-0.047345,-0.446687,-0.004008,-0.062261,-0.359769


In [ ]:
full_x_validation = normalized_test_features.values
full_labels_validation = np.asarray([all_test_sources[id].loc[0, 'category'] for id in all_test_sources])

In [ ]:
# build the model
n_outputs = 2
simple_classifier = tf.keras.Sequential([
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(n_outputs),
])

simple_classifier.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])



In [ ]:
history = simple_classifier.fit(x_train, y_train, epochs = 35, verbose = 2, validation_split = 0.15)

Epoch 1/35
178/178 - 2s - loss: 0.5219 - accuracy: 0.7391 - val_loss: 0.5169 - val_accuracy: 0.7552 - 2s/epoch - 11ms/step
Epoch 2/35
178/178 - 2s - loss: 0.5289 - accuracy: 0.7407 - val_loss: 0.5348 - val_accuracy: 0.7333 - 2s/epoch - 10ms/step
Epoch 3/35
178/178 - 2s - loss: 0.5227 - accuracy: 0.7428 - val_loss: 0.5645 - val_accuracy: 0.7143 - 2s/epoch - 9ms/step
Epoch 4/35
178/178 - 2s - loss: 0.5200 - accuracy: 0.7442 - val_loss: 0.5491 - val_accuracy: 0.7483 - 2s/epoch - 10ms/step
Epoch 5/35
178/178 - 2s - loss: 0.5237 - accuracy: 0.7400 - val_loss: 0.5343 - val_accuracy: 0.7532 - 2s/epoch - 11ms/step
Epoch 6/35
178/178 - 1s - loss: 0.5187 - accuracy: 0.7391 - val_loss: 0.5228 - val_accuracy: 0.7542 - 1s/epoch - 7ms/step
Epoch 7/35
178/178 - 1s - loss: 0.5108 - accuracy: 0.7488 - val_loss: 0.5095 - val_accuracy: 0.7662 - 1s/epoch - 6ms/step
Epoch 8/35
178/178 - 1s - loss: 0.5164 - accuracy: 0.7470 - val_loss: 0.5352 - val_accuracy: 0.7323 - 994ms/epoch - 6ms/step
Epoch 9/35
178/17

In [ ]:
#validate on rest of training data

test_loss, test_acc = simple_classifier.evaluate(x_test,  y_test, verbose=2)

predictions = simple_classifier.predict(x_test)

predicted_class = [np.argmax(pred) for pred in predictions]

total_preds = pd.Series([i for i in range(len(predicted_class))])
preds_to_test = total_preds.sample(10)

for pred in preds_to_test:
  print(f'Predicted class: {predicted_class[pred]}, Real class: {y_test[pred]}')



res = tf.math.confusion_matrix(y_test,predicted_class)
print(res)


37/37 - 0s - loss: 0.5456 - accuracy: 0.7657 - 81ms/epoch - 2ms/step
Predicted class: 1, Real class: 1
Predicted class: 1, Real class: 0
Predicted class: 1, Real class: 0
Predicted class: 0, Real class: 0
Predicted class: 1, Real class: 0
Predicted class: 1, Real class: 1
Predicted class: 1, Real class: 1
Predicted class: 1, Real class: 0
Predicted class: 1, Real class: 1
Predicted class: 0, Real class: 0
tf.Tensor(
[[272 226]
 [ 50 630]], shape=(2, 2), dtype=int32)


In [ ]:
#validate on real test data set

test_loss, test_acc = simple_classifier.evaluate(full_x_validation,  full_labels_validation, verbose=2)

print('\nTest accuracy:', test_acc)


predictions = simple_classifier.predict(full_x_validation)

predicted_class = [np.argmax(pred) for pred in predictions]

total_preds = pd.Series([i for i in range(len(predicted_class))])
preds_to_test = total_preds.sample(10)

for pred in preds_to_test:
  print(f'Predicted class: {predicted_class[pred]}, Real class: {full_labels_validation[pred]}')

res = tf.math.confusion_matrix(full_labels_validation, predicted_class)
print(res)

157/157 - 0s - loss: 1.5823 - accuracy: 0.5260 - 388ms/epoch - 2ms/step

Test accuracy: 0.5260000228881836
Predicted class: 0, Real class: 3
Predicted class: 0, Real class: 0
Predicted class: 0, Real class: 0
Predicted class: 0, Real class: 3
Predicted class: 0, Real class: 0
Predicted class: 11, Real class: 11
Predicted class: 0, Real class: 4
Predicted class: 0, Real class: 0
Predicted class: 0, Real class: 3
Predicted class: 0, Real class: 0
tf.Tensor(
[[2148    0    0   39    3    0   20    1    5    0  118   36    0    3
     0]
 [  55    0    0    4    0    0    0    0    0    0    3    1    0    0
     0]
 [  70    0    0    1    0    0    1    0    0    0    5    1    0    0
     0]
 [1181    0    0   44    6    1   21    0   10    0  110   19    0    3
     0]
 [ 216    0    0   26    3    0    1    1    0    0   12    0    0    0
     0]
 [  38    0    0   10    0    1    2    0    0    0    3    0    0    0
     0]
 [  13    0    0    1    0    0    1    0    0    0    1    